In [1]:
import os
import shutil

In [2]:
train_filenames = os.listdir('kaggle_dogcat/train')
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)

In [3]:
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

rmrf_mkdir('kaggle_dogcat/train2')
os.mkdir('kaggle_dogcat/train2/cats')
os.mkdir('kaggle_dogcat/train2/dogs')

rmrf_mkdir('kaggle_dogcat/test2')
os.symlink('../kaggle_dogcat/test/', 'kaggle_dogcat/test2/test')

for filename in train_cat:
    os.symlink('../../kaggle_dogcat/train/'+filename, 'kaggle_dogcat/train2/cats/'+filename)

for filename in train_dog:
    os.symlink('../../kaggle_dogcat/train/'+filename, 'kaggle_dogcat/train2/dogs/'+filename)

In [4]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

Using TensorFlow backend.


In [9]:
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("kaggle_dogcat/train2", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("kaggle_dogcat/test2", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)

    train = model.predict_generator(train_generator, train_generator.samples)
    test = model.predict_generator(test_generator, test_generator.samples)
    with h5py.File("gap_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

In [10]:
write_gap(ResNet50, (224, 224))

D:\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 0 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


AttributeError: 'function' object has no attribute 'func_name'